In [2]:
import traffic
import pandas as pd
from biblio_herramienta import herramienta

In [3]:
carpetaDatos = r'datos_sectores/'
nombreDatos = "datos_bilbao.csv"

In [4]:
def cargardatos(carpeta,nombre_datos):
    datos_cargados = traffic.core.Traffic.from_file(carpeta+nombre_datos)
    return datos_cargados

In [5]:
vuelos = herramienta.cargardatos(carpetaDatos,nombreDatos)

In [6]:
vuelos.data.head()

,Unnamed: 0,alert,altitude,callsign,geoaltitude,groundspeed,hour,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate
0,0,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:19.800000+00:00,42.334791,-3.236176,False,False,5337.0,2019-02-05 02:10:20+00:00,11.654737,0.0
1,1,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:20.831000064+00:00,42.336467,-3.235665,False,False,5337.0,2019-02-05 02:10:21+00:00,11.654737,0.0
2,2,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:20.831000064+00:00,42.336467,-3.235665,False,False,5337.0,2019-02-05 02:10:22+00:00,11.654737,0.0
3,3,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:22.258999808+00:00,42.338376,-3.235154,False,False,5337.0,2019-02-05 02:10:23+00:00,11.654737,0.0
4,4,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:22.258999808+00:00,42.338376,-3.235154,False,False,5337.0,2019-02-05 02:10:24+00:00,11.654737,0.0


In [7]:
vuelos.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111087 entries, 0 to 2111086
Data columns (total 17 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   alert          bool   
 2   altitude       float64
 3   callsign       object 
 4   geoaltitude    float64
 5   groundspeed    float64
 6   hour           object 
 7   icao24         object 
 8   last_position  object 
 9   latitude       float64
 10  longitude      float64
 11  onground       bool   
 12  spi            bool   
 13  squawk         float64
 14  timestamp      object 
 15  track          float64
 16  vertical_rate  float64
dtypes: bool(3), float64(8), int64(1), object(5)
memory usage: 231.5+ MB


In [9]:
vuelos_prueba = vuelos

In [10]:
def niveldevuelo(altitud_metros):
    try:
        fl = altitud_metros*3.28/1000
        return fl
    except:
        pass
        

In [11]:
vuelos_prueba.data["altitude"] = vuelos_prueba.data["altitude"].apply(niveldevuelo)

In [13]:
vuelos_prueba.data.describe()

,Unnamed: 0,altitude,geoaltitude,groundspeed,latitude,longitude,squawk,track,vertical_rate
count,2.111087e+06,2.066834e+06,1.960798e+06,2.096080e+06,2.111087e+06,2.111087e+06,2.097128e+06,2.096080e+06,2.096922e+06
mean,2.048303e+04,1.058732e+02,3.284019e+04,4.029415e+02,4.320900e+01,-3.652308e+00,4.219430e+03,1.390450e+02,2.075362e+01
std,1.324476e+04,3.410805e+01,9.970326e+03,8.792732e+01,5.719556e-01,9.562478e-01,2.087939e+03,1.137691e+02,7.952514e+02
min,0.000000e+00,-3.280000e+00,-5.250000e+02,4.800000e+01,4.233334e+01,-5.677497e+00,0.000000e+00,0.000000e+00,-6.208000e+03
25%,8.892000e+03,1.092240e+02,3.410000e+04,3.990000e+02,4.270290e+01,-4.416632e+00,2.152000e+03,2.438013e+01,0.000000e+00
50%,1.964300e+04,1.180800e+02,3.657500e+04,4.250000e+02,4.318776e+01,-3.748692e+00,4.662000e+03,1.374091e+02,0.000000e+00
75%,3.083600e+04,1.246400e+02,3.815000e+04,4.540000e+02,4.365159e+01,-2.802600e+00,5.456000e+03,2.141556e+02,0.000000e+00
max,5.577000e+04,4.155760e+02,1.261000e+05,1.025000e+03,4.437583e+01,-2.165840e+00,7.777000e+03,3.598828e+02,3.264000e+04
